In [45]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy
from scipy.linalg import eig


def gen_mask(length, p, prob_func='constant'):
    """
    Call of prob_func gives 1 or 0 with certain probability.
    """
    mask_mat = np.zeros((length, length))
    for i in range(length):
        for j in range(length):
            if i >= j:
                val = 1
                r = np.random.uniform()
                prob = 0
                if prob_func == 'pn':
                    prob = p
                if prob_func == 'log':
                    prob = p * np.log(length) / length
                if prob_func == 'p':
                    prob = p / length
                if r <= prob:
                    val = 0
                mask_mat[i, j] = val
                mask_mat[j, i] = val
    return mask_mat


def eigval_helper(Mat, u=None):
    eigvals, eigvecs = np.linalg.eig(Mat)
    idx = np.argmax(eigvals)
    max_eigvec = eigvecs[:, idx]
    if u is None:
        return sorted(eigvals)
    inner_prod = np.inner(max_eigvec, u.reshape(-1))
    return sorted(eigvals), inner_prod


def sigval_helper(Mat):
    return scipy.linalg.svdvals(Mat)


def get_spectrum(n, theta, mask):
    G = np.random.normal(size=(n, n))
    X = (G + np.transpose(G)) / (2*n)**0.5
    u = np.random.rand(n)
    u = u / np.linalg.norm(u)
    u = u[None, :]
    X_tilda = X + theta * u * u.transpose()
    prev_eigvals, prev_ips = eigval_helper(X_tilda, u)
    X_bar = X_tilda * mask
    # print(mask)
    eigvals, ips = eigval_helper(X_bar, u)
    return prev_eigvals, prev_ips, eigvals, ips, X_bar


def get_multiplied_spectrum(mat_A:np.array):
    Z = gen_mask(mat_A.shape[0], 0.5, 'pn')
    mat_A1 = Z * mat_A
    mat_A2 = np.array(mat_A - mat_A1)
    mat_X = np.matmul(mat_A1, mat_A2.transpose())
    mat_Y = np.matmul(mat_A2, mat_A1.transpose())
    eigvalsX = eigval_helper(mat_X)
    eigvalsY = eigval_helper(mat_Y)
    return eigvalsX, eigvalsY


def get_lnr_eigvec(matrix, return_max=True):
    eigvals1, left_eigvecs = eig(matrix, left=True, right=False)
    _, right_eigvecs = eig(matrix, left=False, right=True)
    if return_max:
        eigval = eigvals1[0]
        left_eigvec = left_eigvecs[0]
        right_eigvec = right_eigvecs[0]
        return np.abs(eigval), left_eigvec, right_eigvec
    return eigvals1, left_eigvecs, right_eigvecs


def get_estimator_P(T):
    """Use the method described in https://arxiv.org/pdf/2005.06062.pdf to get an estimated P (T = P*M). Assuming rank is 1 in this case."""
    Z = gen_mask(T.shape[0], 0.5, 'pn')
    mat_A1 = Z * T
    mat_A2 = np.array(T - mat_A1)
    X = mat_A1 @ mat_A2.transpose()
    Y = mat_A2 @ mat_A1.transpose()
    v, x_, x = get_lnr_eigvec(X)
    eta, pi_, pi = get_lnr_eigvec(Y)
    sigma = np.sqrt(v)
    c1 = np.abs(np.inner(x, x_))**0.5
    c2 = np.abs(np.inner(pi, pi_))**0.5
    print(sigma, c1, c2)
    w = sigma * c1 * c2
    vecx = x[None, :]
    vecy = pi[None, :]
    P = w * (vecx @ vecy.transpose())
    # print(P)
    return P, w


def get_masked_n_unmasked(n, theta, p):
    mask = gen_mask(n, p, 'p')
    prev_eigvals, _, eigvals, _, X_bar = get_spectrum(n, theta, mask)
    P, w = get_estimator_P(X_bar)
    return prev_eigvals, eigvals, P, w 
    

In [46]:
n = 100
theta = 100
p = 0.2

prev_eigvals, eigvals, P, w = get_masked_n_unmasked(n, theta, p)
# print(P)
eigvalP, _, _ = get_lnr_eigvec(P)
print(eigvalP)
print(w)

49.032135000835986 0.5616178855553913 0.39691712080050195
1.1280908145016129
10.930035350056027
